In [228]:
pwd

u'/home/ubuntu/big_mount_point/facebook/scripts'

In [2]:
import numpy as np

In [226]:
import grid_generation as grid
reload(grid)
import feature_engineering as feat
reload(feat)
sub = 'ec2_colsample_bytree0_6_scale_pos_weight1_min_child_weight6_subsample0_9_eta0_1_max_depth3_gamma0_1_th3_n200'
g = grid.Grid(200, 50, 20, 5, pref = 'grid')

In [15]:
feat.generate_test_feature_files(g, sub)

parsing line 1000000
parsing line 2000000
parsing line 3000000
parsing line 4000000
parsing line 5000000
parsing line 6000000
parsing line 7000000
parsing line 8000000

['8607229', '8.1017', '7.8736', '1', '845096']


In [ ]:
data = load_data_from_grid(8, 200)

In [225]:
import xgboost_extra_features_model as xgboost

In [49]:
g = grid.Grid(200, 50, 20, 5, pref = 'grid')
g.generateCardinalityMatrix()

In [223]:
import xgboost_extra_features_model as xgbm
reload(xgbm)
import numpy as np


In [224]:
from sklearn.cross_validation import train_test_split
import xgboost as xgb

In [52]:
data = np.loadtxt('../grid_200_50_20_5/grid_data_0_0.csv', delimiter = ',')

In [53]:
data_features = np.loadtxt('../grid_200_50_20_5_ec2_colsample_bytree0_6_scale_pos_weight1_min_child_weight6_subsample0_9_eta0_1_max_depth3_gamma0_1_th3_n200_features/feature_0_0.csv', delimiter = ',')

In [54]:
print data.shape
print data_features.shape

(2756, 6)
(2756, 131)


In [222]:
data_combined = np.hstack((data[:, :-1], data_features[:, 1:], data[:, -1].reshape(-1, 1)))
mask = np.array(map(lambda x: g.M[0][0][x] > 3, data_combined[:, -1]))
masked_data = data_combined[mask, :]
train, test= train_test_split(masked_data, test_size=0.1)
y_orig, enc = xgbm.trans_y(masked_data[:, -1])
y = enc['encoder'].transform(train[:, -1])[0]
y_test = enc['encoder'].transform(test[:, -1])[0]

In [96]:
params = {
            'objective': 'multi:softprob',
            'eta': 0.01,
            'max_depth': 3,
            'min_child_weight': 6,
            'gamma': 0.1,
            'subsample': 0.9,
            'colsample_bytree': 0.6,
            'scale_pos_weight': 1,
            'nthread': 4,
            'silent': 1,
    'num_class': len(enc['encoder'].classes_)
}

In [221]:
x = xgbm.trans_x(train[:, 1:-1])[0]
x_test = xgbm.trans_x(test[:, 1:-1])[0]
dtrain = xgb.DMatrix(x, label = np.ravel(y))
dtest = xgb.DMatrix(x_test, label = np.ravel(y_test))
bst = xgb.train(params, dtrain, 50, evals = [(dtrain,'train'), (dtest, 'test')], verbose_eval = True, feval = xgbm.map3eval)

XGBoostError: [15:10:10] src/objective/multiclass_obj.cc:43: Check failed: preds.size() == (static_cast<size_t>(param_.num_class) * info.labels.size()) SoftmaxMultiClassObj: label size and pred size does not match

In [104]:
fs = sorted(bst.get_fscore().items(), reverse = True, cmp = lambda x, y:cmp(x[1], y[1]))

In [141]:
x1 = xgbm1.trans_x(train[:, 1:-1])[0]
x_test1 = xgbm1.trans_x(test[:, 1:-1])[0]
dtrain1 = xgb.DMatrix(x1, label = np.ravel(y))
dtest1 = xgb.DMatrix(x_test1, label = np.ravel(y_test))
bst1 = xgb.train(params, dtrain1, 50, evals = [(dtrain1,'train'), (dtest1, 'test')], verbose_eval = True, feval = xgbm.map3eval)

[0]	train-MAP@3:0.611765	test-MAP@3:0.581132
[1]	train-MAP@3:0.614706	test-MAP@3:0.588679
[2]	train-MAP@3:0.628992	test-MAP@3:0.596226
[3]	train-MAP@3:0.638655	test-MAP@3:0.611321
[4]	train-MAP@3:0.646639	test-MAP@3:0.607547
[5]	train-MAP@3:0.659664	test-MAP@3:0.622642
[6]	train-MAP@3:0.660084	test-MAP@3:0.615094
[7]	train-MAP@3:0.661765	test-MAP@3:0.603774
[8]	train-MAP@3:0.671849	test-MAP@3:0.630189
[9]	train-MAP@3:0.676471	test-MAP@3:0.618868
[10]	train-MAP@3:0.681933	test-MAP@3:0.637736
[11]	train-MAP@3:0.681513	test-MAP@3:0.641509
[12]	train-MAP@3:0.686134	test-MAP@3:0.65283
[13]	train-MAP@3:0.689496	test-MAP@3:0.65283
[14]	train-MAP@3:0.692017	test-MAP@3:0.65283
[15]	train-MAP@3:0.691176	test-MAP@3:0.656604
[16]	train-MAP@3:0.691597	test-MAP@3:0.656604
[17]	train-MAP@3:0.693697	test-MAP@3:0.656604
[18]	train-MAP@3:0.695378	test-MAP@3:0.664151
[19]	train-MAP@3:0.696639	test-MAP@3:0.664151
[20]	train-MAP@3:0.698739	test-MAP@3:0.660377
[21]	train-MAP@3:0.70042	test-MAP@3:0.664151
[2

In [139]:
def trans_x2(X, x_transformer = None):
    """
    X = [[x, y, a, t, feats]]
    """
    fw = [1., 1., 1., 1., 1., 1., 1.]
    minute_v = X[:, 3]%60
    hour_v = X[:, 3]//60
    weekday_v = hour_v//24
    month_v = weekday_v//30
    year_v = (weekday_v//365 + 1)*fw[5]
    hour_v = ((hour_v%24 + 1) + minute_v/60.0)*fw[2]
    hour_v_2 = (X[:, 3]%(60*60*24))//(60*60*2)
    hour_v_3 = (X[:, 3]%(60*60*24))//(60*60*3)
    hour_v_4 = (X[:, 3]%(60*60*24))//(60*60*4)
    hour_v_6 = (X[:, 3]%(60*60*24))//(60*60*6)
    hour_v_8 = (X[:, 3]%(60*60*24))//(60*60*8)
    weekday_v = (weekday_v%7 + 1)*fw[3]
    month_v = (month_v%12 +1)*fw[4]
    accuracy_v = np.log10(X[:, 2])*fw[6]
    x_v = X[:, 0]*fw[0]
    y_v = X[:, 1]*fw[1]
    
    x_h = np.median(X[:, range(4, 134, 13)], axis = 1).reshape(-1, 1)
    x_h2 = np.median(X[:, range(5, 134, 13)], axis = 1).reshape(-1, 1)
    x_h3 = np.median(X[:, range(6, 134, 13)], axis = 1).reshape(-1, 1)
    x_h4 = np.median(X[:, range(7, 134, 13)], axis = 1).reshape(-1, 1)
    x_h6 = np.median(X[:, range(8, 134, 13)], axis = 1).reshape(-1, 1)
    x_h8 = np.median(X[:, range(9, 134, 13)], axis = 1).reshape(-1, 1)
    x_week = np.median(X[:, range(10, 134, 13)], axis = 1).reshape(-1, 1)
    x_month = np.median(X[:, range(11, 134, 13)], axis = 1).reshape(-1, 1)
    x_count = np.log10(np.median(X[:, range(12, 134, 13)], axis = 1)).reshape(-1, 1)
    x_a_med = np.log10(np.median(X[:, range(13, 134, 13)], axis = 1)).reshape(-1, 1)
    x_x_med = np.median(X[:, range(14, 134, 13)], axis = 1).reshape(-1, 1)
    x_y_med = np.median(X[:, range(15, 134, 13)], axis = 1).reshape(-1, 1)
    x_abs_distance = np.median(X[:, range(16, 134, 13)], axis = 1).reshape(-1, 1)
    median_stats = np.hstack((x_h, x_h2, x_h3, x_h4, x_h6, x_h8, x_week, x_month, x_count, x_a_med, x_x_med, x_y_med, x_abs_distance))
    print median_stats.shape
    print x_v.shape
    X_new = np.hstack((x_v.reshape(-1, 1),\
                     y_v.reshape(-1, 1),\
                     accuracy_v.reshape(-1, 1),\
                     hour_v.reshape(-1, 1),\
                     hour_v_2.reshape(-1, 1),\
                     hour_v_3.reshape(-1, 1),\
                     hour_v_4.reshape(-1, 1),\
                     hour_v_6.reshape(-1, 1),\
                     hour_v_8.reshape(-1, 1),\
                     weekday_v.reshape(-1, 1),\
                     month_v.reshape(-1, 1),\
                     year_v.reshape(-1, 1),\
                     median_stats))

    return (X_new, x_transformer)


In [140]:
x2 = trans_x2(train[:, 1:-1])[0]
x_test2 = trans_x2(test[:, 1:-1])[0]
dtrain2 = xgb.DMatrix(x2, label = np.ravel(y))
dtest2 = xgb.DMatrix(x_test2, label = np.ravel(y_test))
bst3 = xgb.train(params, dtrain2, 50, evals = [(dtrain2,'train'), (dtest2, 'test')], verbose_eval = True, feval = xgbm.map3eval)

(2380, 13)
(2380,)
(265, 13)
(265,)
[0]	train-MAP@3:0.651261	test-MAP@3:0.588679
[1]	train-MAP@3:0.648319	test-MAP@3:0.581132
[2]	train-MAP@3:0.654622	test-MAP@3:0.573585
[3]	train-MAP@3:0.656723	test-MAP@3:0.584906
[4]	train-MAP@3:0.662185	test-MAP@3:0.592453
[5]	train-MAP@3:0.668067	test-MAP@3:0.592453
[6]	train-MAP@3:0.67395	test-MAP@3:0.603774
[7]	train-MAP@3:0.678992	test-MAP@3:0.622642
[8]	train-MAP@3:0.681933	test-MAP@3:0.622642
[9]	train-MAP@3:0.682773	test-MAP@3:0.630189
[10]	train-MAP@3:0.684034	test-MAP@3:0.630189
[11]	train-MAP@3:0.686555	test-MAP@3:0.633962
[12]	train-MAP@3:0.687815	test-MAP@3:0.637736
[13]	train-MAP@3:0.691176	test-MAP@3:0.630189
[14]	train-MAP@3:0.691597	test-MAP@3:0.637736
[15]	train-MAP@3:0.694538	test-MAP@3:0.637736
[16]	train-MAP@3:0.694118	test-MAP@3:0.630189
[17]	train-MAP@3:0.697059	test-MAP@3:0.637736
[18]	train-MAP@3:0.69916	test-MAP@3:0.637736
[19]	train-MAP@3:0.701261	test-MAP@3:0.641509
[20]	train-MAP@3:0.702521	test-MAP@3:0.645283
[21]	train

In [144]:
sorted(bst.get_fscore().items(), reverse = True, cmp = lambda x, y:cmp(x[1], y[1]))[:10]

[('f22', 341),
 ('f20', 326),
 ('f24', 322),
 ('f12', 206),
 ('f23', 198),
 ('f1', 196),
 ('f35', 179),
 ('f36', 162),
 ('f21', 157),
 ('f37', 147)]

In [202]:
data_combined = np.hstack((data[:, :-1], data_features[:, 1:], data[:, -1].reshape(-1, 1)))
mask4 = np.array(map(lambda x: g.M[0][0][x] > 10, data_combined[:, -1]))
masked_data4 = data_combined[mask4, :]
train4, test4= train_test_split(masked_data4, test_size=0.1)
y_orig4, enc4 = xgbm.trans_y(masked_data4[:, -1])
y4 = enc4['encoder'].transform(train4[:, -1])
y_test4 = enc4['encoder'].transform(test4[:, -1])
params['num_class'] = len(enc4['encoder'].classes_)

x4 = trans_x2(train4[:, 1:-1])[0]
x_test4 = trans_x2(test4[:, 1:-1])[0]


(2286, 13)
(2286,)
(254, 13)
(254,)


In [220]:
from sklearn.naive_bayes import GaussianNB
import helpers

In [203]:
dtrain4 = xgb.DMatrix(x4, label = np.ravel(y4))
dtest4 = xgb.DMatrix(x_test4, label = np.ravel(y_test4))
bst4 = xgb.train(params, dtrain4, 50, evals = [(dtrain4,'train'), (dtest4, 'test')], verbose_eval = True, feval = xgbm.map3eval)
bst_map = mapk(bst4.predict(dtest4), y_test4, enc4)

[0]	train-MAP@3:0.656605	test-MAP@3:0.637795
[1]	train-MAP@3:0.686789	test-MAP@3:0.65748
[2]	train-MAP@3:0.692038	test-MAP@3:0.669291
[3]	train-MAP@3:0.692913	test-MAP@3:0.681102
[4]	train-MAP@3:0.701225	test-MAP@3:0.681102
[5]	train-MAP@3:0.708661	test-MAP@3:0.688976
[6]	train-MAP@3:0.712598	test-MAP@3:0.69685
[7]	train-MAP@3:0.71741	test-MAP@3:0.708661
[8]	train-MAP@3:0.720472	test-MAP@3:0.712598
[9]	train-MAP@3:0.724409	test-MAP@3:0.708661
[10]	train-MAP@3:0.726159	test-MAP@3:0.716535
[11]	train-MAP@3:0.727034	test-MAP@3:0.708661
[12]	train-MAP@3:0.726159	test-MAP@3:0.712598
[13]	train-MAP@3:0.730096	test-MAP@3:0.724409
[14]	train-MAP@3:0.731409	test-MAP@3:0.724409
[15]	train-MAP@3:0.734471	test-MAP@3:0.724409
[16]	train-MAP@3:0.734908	test-MAP@3:0.724409
[17]	train-MAP@3:0.738408	test-MAP@3:0.724409
[18]	train-MAP@3:0.741032	test-MAP@3:0.728346
[19]	train-MAP@3:0.739283	test-MAP@3:0.724409
[20]	train-MAP@3:0.741032	test-MAP@3:0.724409
[21]	train-MAP@3:0.742345	test-MAP@3:0.724409
[

In [204]:
gnb = GaussianNB()
gnb.fit(x4, y4)
gnb_preds = gnb.predict_proba(x_test4)
gnb_map = mapk(gnb_preds, y_test4, enc4)

rfc = RandomForestClassifier()
rfc.fit(x4, y4)
rfc_preds = rfc.predict_proba(x_test4)
rfc_map = mapk(rfc_preds, y_test4, enc4)

abc = AdaBoostClassifier()
abc.fit(x4, y4)
abc_preds = abc.predict_proba(x_test4)
abc_map = mapk(abc_preds, y_test4, enc4)

In [201]:
def mapk(probs, y, enc):
    y_places = enc['encoder'].inverse_transform(np.argsort(probs, axis = 1)[:, ::-1][:, :3])
    a_places = enc['encoder'].inverse_transform(y).reshape(-1, 1)
    rows = np.hstack((y_places, a_places))
    m = np.mean(map(lambda row: helpers.apk(row[-1:], row[:-1]), rows))
    return m

In [209]:
print bst_map, gnb_map, rfc_map, abc_map

0.737532808399 0.660104986877 0.71062992126 0.406167979003


In [217]:
rfc1 = RandomForestClassifier(n_estimators=30, max_depth = 13, max_features='log2')
rfc1.fit(x4, y4)
rfc_preds1 = rfc.predict_proba(x_test4)
rfc_map1 = mapk(rfc_preds1, y_test4, enc4)

In [218]:
rfc_map1

0.71062992125984248

In [208]:
abc_map

0.40616797900262464

In [219]:
import numpy as np

In [229]:
cv_top_t = np.loadtxt('../grid_200_50_20_5_ec2_colsample_bytree0_6_scale_pos_weight1_min_child_weight6_subsample0_9_eta0_1_max_depth3_gamma0_1_th3_n200/cv_top_t.csv', delimiter = ',', dtype = int)

In [230]:
cv_ac = np.loadtxt('../main_cv_0.02_5.csv', delimiter = ',', dtype = int, usecols = (0, 5))

In [232]:
ranks = []
for i in range(len(cv_ac)):
    a_p = cv_ac[i][1]
    preds = list(cv_top_t[i])
    if preds.count(a_p) == 0:
        ranks.append(0)
    else:
        ranks.append(preds.index(a_p))

In [233]:
ranks.count(0)/float(len(ranks))

0.13196571622947242

In [234]:
ranks.count(1)/float(len(ranks))

0.5930683424872398

In [235]:
ranks.count(1)/float(len(ranks))

0.5930683424872398

In [237]:
for i in range(10):
    print i, ranks.count(i)/float(len(ranks))

 0 0.131965716229
1 0.593068342487
2 0.123868932781
3 0.0533566518864
4 0.0302826229249
5 0.0203922808259
6 0.0141612106864
7 0.0110018674313
8 0.00861904573746
9 0.00716220882589


In [238]:
helpers.apk([1], [1,2,3])

1.0

In [239]:
helpers.apk([2], [1, 2, 3])

0.5

In [240]:
helpers.apk([3], [1,2,3])

0.3333333333333333

In [247]:
apk_dict = {1: 1.0/1.0, 2: 1.0/2.0, 3: 1.0/3.0}

def get_apk_dict(s):
    if s in apk_dict:
        return apk_dict[s]
    else:
        return 0.0

In [244]:
apk_dict[1]

1.0

In [248]:
m = np.mean(map(get_apk_dict, ranks))

In [249]:
m

0.67278835950657656

In [252]:
import copy
ranks1 = copy.copy(ranks)

In [256]:
for i in range(len(ranks)):
    if ranks1[i] == 2:
        ranks1[i] = 1

In [257]:
np.mean(map(get_apk_dict, ranks1))

0.73472282589710847

In [255]:
for i in range(10):
    print i, ranks1.count(i)/float(len(ranks))

0 0.131965716229
1 0.593068342487
2 0.123868932781
3 0.0533566518864
4 0.0302826229249
5 0.0203922808259
6 0.0141612106864
7 0.0110018674313
8 0.00861904573746
9 0.00716220882589


In [259]:
import numpy as np

In [261]:
import scipy as sp

In [264]:
m = sp.stats.mode([1,1, 2, 3])

In [274]:
sp.stats.mode([[1,2,3], [1,1,2], [2,1,3], [2, 3, 2]])

ModeResult(mode=array([[1, 1, 2]]), count=array([[2, 2, 2]]))

In [275]:
np.median([[1,2,3], [1,1,2], [2,1,3], [2, 3, 2]])

2.0

In [277]:
sp.stats.mode([1, 1, 2, 2])

ModeResult(mode=array([1]), count=array([2]))

In [278]:
import grid_generation as grid
import feature_engineering as feat
g = grid.Grid(200, 50, 20, 5, pref = 'grid')
submission_name = 'grid_200_50_20_5_def_params_th7_n100'

In [280]:
top_t_cv = np.loadtxt(g.getTopPlacesFolder(submission_name) + 'cv_top_t.csv', delimiter = ',', dtype = int)

In [284]:
ranks = []
for i in range(len(cv_ac)):
    a_p = cv_ac[i][1]
    preds = list(top_t_cv[i])
    if preds.count(a_p) == 0:
        ranks.append(0)
    else:
        ranks.append(preds.index(a_p))

In [285]:
for i in range(10):
    print i, ranks.count(i)/float(len(ranks))

0 0.134001508376
1 0.591561684502
2 0.124068217076
3 0.054018069588
4 0.0303032385416
5 0.0200126098855
6 0.0143965723101
7 0.0107665058076
8 0.0084335051874
9 0.00682376911884


In [286]:
np.mean(map(get_apk_dict, ranks))

0.67160181623582893